### Explore these logs of access to Codeup Curriculum: Ask pointed questions utilizing the Data Science Pipeline and see what you can find that interests you about this data!

Ideas:
- Can you label students who are viewing both the web dev and data science curriculum?
- Can you label students by the program they are in?
- Can you label users by student vs. staff?
- What are Zach, Maggie, Faith, and Ryan's ids?